<a href="https://colab.research.google.com/github/patience275/recipebot/blob/main/recipie_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-core  --quiet

In [2]:
!pip install -qU 'langchain[mistralai]'

In [3]:
import getpass
import os

if not os.environ.get('MISTRAL_API_KEY'):
  os.environ['MISTRAL_API_KEY'] = getpass.getpass('enter api key for mistral: ')

enter api key for mistral: ··········


In [4]:
from langchain.chat_models import init_chat_model
llm=init_chat_model('mistral-small-latest',model_provider='mistralai')

from langchain_core.messages import HumanMessage
llm.invoke([HumanMessage(content='hi, im colleen')])

AIMessage(content='Hi, Colleen! 😊 How can I help you today? Let me know if you have any questions or need assistance with something!', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 8, 'total_tokens': 38, 'completion_tokens': 30}, 'model_name': 'mistral-small-latest', 'model': 'mistral-small-latest', 'finish_reason': 'stop'}, id='run--29b6b5e5-2be7-4911-a1fa-ba5f8594004c-0', usage_metadata={'input_tokens': 8, 'output_tokens': 30, 'total_tokens': 38})

In [5]:
from langchain.memory import ConversationSummaryMemory
memory=ConversationSummaryMemory(llm=llm, return_messages=True,memory_key='chat_history')

/tmp/ipython-input-346278106.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationSummaryMemory(llm=llm, return_messages=True,memory_key='chat_history')


In [6]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate([
    ('system','you are an excellent chef. using the {ingredients} provided by the user, create a delicious meal. include a catchy name, utensils to use, and the steps to follow to create the meal you suggest'),
    MessagesPlaceholder(variable_name ='chat_history'),
     ('user','{ingredients}')
])

In [7]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

chain=RunnableParallel(
    chat_history=lambda x:memory.chat_memory.messages,
    ingredients=RunnablePassthrough() # Change key to 'ingredients' to match prompt
) | prompt | llm

In [8]:
while True:
  user_ingredients=input('n\ enter your ingredients(or type"exit" to quit)')
  if user_ingredients.lower()=='exit':
    break
  response=chain.invoke(input={'ingredients':user_ingredients})
  print(response)

<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipython-input-1880630427.py:2: SyntaxWarning: invalid escape sequence '\ '
  user_ingredients=input('n\ enter your ingredients(or type"exit" to quit)')


n\ enter your ingredients(or type"exit" to quit)onions,rice, tomatoes
content='**Dish Name:** "Tasty Tomato Rice Pilaf"\n\n**Utensils:**\n1. Medium-sized pot with a lid\n2. Cutting board\n3. Knife\n4. Wooden spoon\n5. Measuring cups and spoons\n6. Colander (for rinsing rice, if necessary)\n\n**Steps:**\n\n1. **Prepare the rice:**\n   - Rinse 1 cup of rice under cold water using a colander to remove excess starch. Set aside.\n\n2. **Chop the vegetables:**\n   - Finely chop 1 medium onion.\n   - Dice 2 medium tomatoes.\n\n3. **Cook the onions:**\n   - Heat 2 tablespoons of oil in the pot over medium heat.\n   - Add the chopped onions and sauté until they become translucent and slightly golden, about 5 minutes.\n\n4. **Add the tomatoes:**\n   - Stir in the diced tomatoes and cook for about 5 minutes, until they soften and start to break down.\n\n5. **Combine the rice:**\n   - Add the rinsed rice to the pot and stir well to combine with the onion and tomato mixture.\n   - Cook for 2-3 minu

In [9]:
!pip install gradio --quiet

In [10]:
def generate_recipie(ingredients):
  return chain.invoke({'ingredients': ingredients}).content # Access the content attribute

import gradio as gr
chatbot=gr.Interface(
 fn=generate_recipie,
 inputs=gr.Textbox(lines=4,placeholder='e.g:tomatoes,onions,eggs', label='your ingredients'), # label moved inside gr.Textbox
 outputs=gr.Textbox(label='generated recipie'), # label moved inside gr.Textbox
 title='your awesome recipie generator',
 description='eneter your ingredients ypu have  and get a delicious recipie,tyhe utensils to use and step by step instructions'
)
chatbot.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8140f8e1c6e01d1801.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
%%writefile recipie.py
import getpass
import os

if not os.environ.get('MISTRAL_API_KEY'):
  os.environ['MISTRAL_API_KEY'] = getpass.getpass('enter api key for mistral: ')

from langchain.chat_models import init_chat_model
llm=init_chat_model('mistral-small-latest',model_provider='mistralai')


from langchain.memory import ConversationSummaryMemory
memory=ConversationSummaryMemory(llm=llm, return_messages=True,memory_key='chat_history')

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate([
    ('system','you are an excellent chef. using the {ingredients} provided by the user, create a delicious meal. include a catchy name, utensils to use, and the steps to follow to create the meal you suggest'),
    MessagesPlaceholder(variable_name ='chat_history'),
     ('user','{ingredients}')
])

from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

chain=RunnableParallel(
    chat_history=lambda x:memory.chat_memory.messages,
    ingredients=RunnablePassthrough() # Change key to 'ingredients' to match prompt
) | prompt | llm

import gradio as gr
def generate_recipie(ingredients):
  return chain.invoke({'ingredients': ingredients}).content # Access the content attribute

import gradio as gr
chatbot=gr.Interface(
 fn=generate_recipie,
 inputs=gr.Textbox(lines=4,placeholder='e.g:tomatoes,onions,eggs', label='your ingredients'), # label moved inside gr.Textbox
 outputs=gr.Textbox(label='generated recipie'), # label moved inside gr.Textbox
 title='your awesome recipie generator',
 description='enter your ingredients you have  and get a delicious recipie,tyhe utensils to use and step by step instructions'
)
chatbot.launch()


Writing recipie.py


In [12]:
%%writefile requirements.txt
gradio
langchain-core
langchain
mistralai

Writing requirements.txt
